# 1 - Introduction

    This project was inspired by the 2021 paper, "Searching for young stellar objects through SEDs by machine learning", by Chiu et. al. In it, the researchers train a machine learning model (a neural network) on the Spitzer From Molecular Cores to Planet-Forming Disks (C2D) dataset. They attempted to successfully classify 3 general types of object; stars, young stellar objects (YSOs), and galaxies. This machine learning classification will aid the identification of YSOs, as a major issue in the contemporary field is that YSO classification is somewhat demanding in both manual effort, as well as quite uncertain due to the connection between YSOs and stellar sources. If we are able to identify where, in the parameter space of common measured flux bands, YSOs are typically located; we can be more confident and efficient in classification of other sources with potential to be YSOs.
    We must note that the spectral features of YSOs are what enables this. As the delivery document from the C2D dataset (Evans et. al. 2014) states, the main quality of a YSO is the presence of a significant dust component, which appears as an excess in infrared wavelengths. Comparably, stars can be modelled by a stellar photosphere, which generally acts as a black-body, albeit with some reddening from ISM extinction. What this means is that, if we are to take data ranging from the near to far-IR, just as the Spitzer C2D dataset provides, we can make conclusions on an object's type based upon the ratio of flux between shorter and longer wavelengths. This is the basis of manual classification as was used in the C2D dataset. But of course, with machine learning, we need not manually evaluate this; we can treat every flux band as a 'dimension', and identify the 'region' in which YSOs are congregated; allowing for greater applicability to different datasets.
    The Spitzer From Molecular Cores to Planet-Forming Disks (C2D) dataset used Spitzer to target both a variety of known YSOs, as well as all sources in five of the nearest large molecular clouds. The intention was to form a dataset which represents not only our best known examples of YSOs, but also which describes the environments in which these YSOs exist. The most important features of the dataset were the four IRAC bands and MIPS 1 band, corresponding to fluxes at 3.6, 4.5, 5.8, 8.0, and 24 microns, as well as the object_type, which was the manual label of each object as described in the delivery document. Note that the correct subset of the dataset is the '07 Full CLOUDS catalog, with 4.2 million objects, and contains all objects registered in those aforementioned molecular clouds. 
    In terms of data exploration, several steps were necessary before any machine learning took place. Firstly, an SQL query was given when retrieving the data, specifying that only sources without NULL in the 5 bands and object_type feature, were allowed to be given. This reduced the dataset immediately from 4.2 million to approximately 2.2 million objects. Secondly, upon reading the data in, I noticed that many objects that were not literally labelled as NULL, had flux values that ranged from 0 to negative values. Zero values are not ideal; I trimmed those from the dataset, as I desired only datapoints which at least some quantifiable flux. The negative flux values were, and are, extremely puzzling to me. Keep in mind some of these negative values were as large as -2 mJy. Given that "negative" flux is unphysical, I can only assume that it was the result of some form of bias-correction or photometric correction. However, no part of the delivery document indicates that negative values could arise. But, finding the sources of these data is not my prerogative; thus, I also cut these out, resulting in a final count of ~598,000 objects.

# 2 - Approach

    My chosen algorithm will be Support Vector Machines (SVM). This is due to two simple aspects; firstly, as we've demonstrated before, the classes should spatially separate when put into the 5-dimensional "flux" parameter space, which will hopefully allow for hyperplanes or other kernel shapes to "cut up" our classes. Secondly, it is one of the more simple and reliable machine learning algorithms, and I typically value these over others. SVMs can especially be contrasted to the methods of the inspiratory paper, Chiu et. al., and their use of a somewhat opaque neural network. 
    As stated in the introduction, our main features will be the IRAC 1-4 bands, and MIPS 1 band, while our labels are described by the object_type. Note that object_type is in itself a complicated part of the dataset. It is not simply "star, galaxy, YSO", but instead defined by 33 detailed subtypes. More details can be found in the relevant section of the 2014 delivery document, but as a brief overview, the subtypes can generally be separated into four larger classifications. Subtype strings begin with their "root" classification, i.e. "star", "YSOc", and "Galc", thus we can easily group these together into three main classifications. All other labels can be generally grouped together into a fourth class, which we immediately discard. This includes several subtypes explicitly used for inconclusive classifications, objects with missing data, or simply unknown objects. Thus, after this processing, we return to a simple three-class system. This gives us a final count of ~454k "discards", ~140k stars, ~2,900 galaxies, and only ~1,000 YSOs.
    When it comes to our model parameters, I have decided to forego hyperparameter optimization due to its inevitable computational complexity, and my desire for a certain level of simplicity in this project, as well as my own biases. If necessary, future implementations could explore different hyperparameters, but as we shall see later, my current level of success gives me no desire to do so. Thus, when using scikit's SVC Support Vector Machine implementation, I select the default C parameter of regularization (1.0), as well as the default Radial Basis Function (RBF) kernel. Qualitatively, this kernel is well-suited for the dataset; this is because two of our classes, YSOs and galaxies, are somewhat blobby (and thus a linear kernel would struggle).
    In terms of assessment metrics, I found that traditional training/testing metrics are essentially useless for such an unbalanced dataset, as approximately 97% of our dataset consists of stars. So, as a thought experiment, we could mindlessly classify everything as a stellar source and still find "amazing" training and testing statistics. A more useful metric is a normalized confusion matrix, in which we normalize over the "true" labelling. This allows us to immediately see what proportion of the smaller classes is correctly classified.

# 3 - Initial Results

    Firstly, there is one, somewhat concerning aspect. As has been mentioned before, this project is inspired by, and intends to reproduce, some of the aspects of the 2021 Chiu et. al. paper. When initially attempting to reproduce figures from their work, I noted significant discrepancies between their work and my own, even though we source our data from the same set. Compare the two following figures; Chiu's Fig. 4(a), which is a log-log-log plot of the IRAC 3, IRAC 4, and MIPS 1 bands, and my own attempt to reproduce the figure with the same bounds on X, Y, and Z.

![chiu4a](chiu4a.png) 

![4acomp](4acomp.png) 

    Note that while the YSO and galaxy populations are seemingly identical, my population of stars are much more numerous and extend to much smaller fluxes than compared to Fig. 4(a), which seemingly truncates the majority of the stars past a certain flux. As of now, I have no answer for this discrepancy; unfortunately, I have not had time to contact the researchers. I have reviewed the paper extensively and, as a layman, cannot identify any obvious clarifications of what may have caused this. Chiu et. al.'s data preparation is essentially similar to mine; removal of non-detections as the key point. The only culprit I can identify is what the researchers describe as removal of bias due to non-detections. Per my understanding, they noted that extremely small flux values are more likely complete non-detections with a small degree of photon flux/thermal noise being misidentified as meaningful information. Thus they replace these values with a hundredth of the "reliable" flux of that band. However, I do not think this is the cause; firstly, they are altering data, not removing it; so this does not explain the numerical discrepancy between my count (~140k) and their count (~59k) of identified stars. Secondly, they explicitly highlight that this change is small and has no effect on testing or training results. 
    Other possible answers include adjustments for extinction, or perhaps a simple step so elementary that it was deemed unnecessary to elaborate upon, and yet unknown to a layman such as I. It could also be that the researchers had similar problems with an unbalanced dataset, and thus removed a large proportion of stellar objects; but this would need to be done randomly/equally, to not introduce bias. I also considered that this was an artifact of stricter object_type grouping; I ran a modified version of my code which discarded everything not explicitly labeled as the three main classes (i.e. discarded all subtypes) and could not reproduce the discrepancy. I would most likely conclude that the researchers simply 'cut out' stars based on if they undershot a given flux value in one or more bands. This could be a viable selection method, but if so, it is necessary to explain it. I am concerned by the fact that I see no clear explanation. 
    Below, I also compare the researchers' corner plot, with accompanying histograms, to my own normalized histogram of those same three bands, for a more quantitative comparison at the differences between our data.

![chiucorner](chiucorner.png)

![hists](hists.png)

    While it's impossible to say for certain, because the original plot is also normalized, this comparison makes it seem as if it is not an explicit 'cutting out' of stellar sources, but also a shift; as the researchers' version also preserves an overall Gaussian-ish natural shape, rather than a cliff that would arise from a 'cut'. Perhaps both a removal of points, and a shift in the form of the aforementioned 'non-detection bias correction', could be at play. Regardless, this issue will have no impact on the continuation of the project.
    As part of the initial investigation, I applied an extremely simple linear SVM using scikit's LinearSVC. This allowed for a simple visual comparison between the original classification and a basic classification using a linear kernel. This was more of a trial run to see if the methods and code actually functioned, which they did.

![basicORIG](basicORIG.png)

![basicLINSVM](basicLINSVM.png)

    This test gave results as expected; the SVM generally struggles where the classes are intermixed finely, or in the cases of outliers, but correctly identified the general regions.
    As a more advanced initial trial, I applied SVm with an RBF kernel and straitified 5-fold, in order to evaluate the training and testing scores. These scored were both in excess of 99.5%, again due to the aforementioned fact that 97% of our dataset is of a single class. A confusion matrix was a more appropriate form of evaluation, and thus I present one with normalization relative to the total number of data points.

![confmatRBFnormall](confmatRBFnormall.png)

    As we can see, the domination of the stellar classification makes this form of confusion matrix redundant. What is more useful, is normalizing over the "true" label. This allows us to see the fraction of each class that has been classified correctly.

![confmatRBFnormtrue](confmatRBFnormtrue.png)

    Here, we can see that galaxies and YSOs have success rates of around 90%, which is fairly high considering the dominance of the stellar class and the potential for misclassification in the "mixed" regions of both stars and other objects. Overall, however, there are two main steps that need to be taken. Firstly, we should attempt dimensionality reduction through PCA; as one can see in both of the above scatter and corner plots, certain variables (such as IRAC 3 and IRAC 4) have a fairly linear relation, and thus have a high potential for useful dimensional reduction. And if we can apply dimensionality reduction, this would allow our other algorithm, SVM, to be more efficient due to the decrease in complexity. Secondly, we should attempt some form of weighing when it comes to the importance of each class. We want to correctly classify as many YSOs as possible, but misclassification of stars is not a major concern; thus we could apply an inverse weight which prioritizes the smaller classes.

# 4 - Final Results and Reflection

    The main form of optimization I focused on is dimensionality reduction through PCA, for the aforementioned reasons. Firstly, we can look at the dimensionality reduction applied to the original classification. Note that I leave the axes unlabelled as they are now somewhat disconnected principal components, not literal measurements. The same overall shape we could infer from the IRAC3-IRAC4-MIPS1 scatter plot is still present, despite the fact that we've reduced from 5 dimensions, which includes the IRAC 2 and 3 bands.

![PCAorig](PCAorig.png)

    And here we have a simple RBF kernel applied to the reduced data. Note that, as mentioned before, the sheer number of stellar points tends to overpower the other two classifications in the 'border' zones. 

![PCArbf](PCArbf.png)

    This confusion matrix quantitatively supports this analysis; note how the rate of true classifications for YSOs and galaxies has dropped into the mid-80%s, while the number of misclassifications labelling them as stars is now significant. Obviously, this is a major indication that weighing is needed, in order to prioritize the classification of galaxies and YSOs.

![PCArbfconf](PCArbfconf.png)

    Therefore, we can redo our RBF kernel SVM, applied to the full 5-dimensional dataset, but with a "balanced" class weighing which automatically adjust weights inversely proportional to class frequencies in the input data. I present a similar 3D scatter plot, but this figure is not illuminating on its own.

![weightedRBF](weightedRBF.png)

    What is more interesting is the corresponding confusion matrix, which shows that correct classifications have reached extremely high values, in excess of 98%, for all three classes. Essentially, when we sacrifice the few stars in the border regions by decreasing the importance of stellar weighing, all classifications increase simply because the stellar class can 'afford' to be misclassified. 

![weightedRBFconf](weightedRBFconf.png)

    We can also apply this weighing to the reduced data, and visually see how the "strengths" of the galaxy and YSO classification push the borders in this two-dimensional presentation. 

![weightedPCArbf](weightedPCArbf.png)

    And again, the confusion matrix itself is more illuminating. All three "true-true" classifications are at 96% or higher success, and now a degree of mutual competiton has actually arose between galaxy and YSO classification. But again, PCA is essentially a compromise; our full 5-dimensional SVM treatment still has a greater performance.

![weightedPCArbfconf](weightedPCArbfconf.png)

    For this particular dataset and problem, I believe that SVM is one of the best choices for classification. At the same time, I do note that the circumstances of the data tell us that any classification algorithm that can either be applied to the PCA'd data, or work in multiple dimensions (such as kNN), could achieve similar results. After all, in the end, this is a very simplistic case of classes which dominate well-defined regions; the only major complication is the imbalanced dataset, which we have mitigated through class weighing which is common among similar algorithms. Thus, while SVM performed well, a variety of other approaches could be equally successful.
    Unfortunately, one aspect of this project I did not reach was the potential to apply these methods on the Spitzer Enhanced Imaging Products (SEIP) dataset. This would have allowed us to determine whether the algorithm, trained on a small subset of data in the form of the C2D set, which focuses on molecular clouds, could also function on the dataset containing the majority of Spitzer products, and thus acting as a much more general sample of the universe. I am simply limited by time and the space on my hard drive; I would emphasize, however, that it would be a simple matter of importing the dataset and selecting out the 5 bands before proceeding as normal. 

# 5 - Link to Github repo

GITHUB: https://github.com/AuthmaStasley/A416FinalProject

# 6 - AI Statement

    No AI was used at any stage in this project; all code that did not organically spring from my mind or from following our in-class notebooks, or scikit documentation, was written using the assistance of ancient posts on StackOverflow. 

# 7 - Resources

    Link to Spitzer C2D dataset: https://irsa.ipac.caltech.edu/data/SPITZER/C2D/overview.html

    "Searching for young stellar objects through SEDs by machine learning" by Chiu et. al., 2021, can be found here: https://ui.adsabs.harvard.edu/abs/2021A%26C....3600470C/abstract

    "Final delivery of data from the C2D Legacy Project" by Evans et. al., 2014, can be found here: https://irsa.ipac.caltech.edu/data/SPITZER/C2D/doc/c2d_del_document.pdf (referenced as "the delivery document" in this report)

    "From Molecular Cores to Planet-forming Disks: An SIRTF Legacy Program" by Evans et. al., 2003, can be found here: https://ui.adsabs.harvard.edu/abs/2003PASP..115..965E/abstract

    "The Spitzer c2d Legacy Results: Star-Formation Rates and Efficiencies; Evolution and Lifetimes" by Evans et. al., 2009, can be found here: https://ui.adsabs.harvard.edu/abs/2009ApJS..181..321E/abstract